# Calculation of sentiment score for Disney fandom

The code in this notebook is used for calculating the sentiment scores of the Disney fandom pages.

In [1]:
# Import packages
import urllib.request 
import json 
import re 
import math
import os
from urllib.parse import quote
import nltk, pprint
import numpy as np
import matplotlib.pyplot as plt

In [11]:
#Sentiment functions

#set a tokenizing pattern
def my_tokenize(text):
    pattern = r'''(?x)     # set flag to allow verbose regexps
        (?:[A-Z]\.)+       # abbreviations, e.g. U.S.A.
      | \w+(?:-\w+)*       # words with optional internal hyphens
      | \$?\d+(?:\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
      | \.\.\.             # ellipsis
      | [][.,;"'?():-_`]   # these are separate tokens; includes ], [
    '''
    return nltk.regexp_tokenize(text, pattern)

def tokenize_and_lower(raw):
    tokens=my_tokenize(raw)
    cleaned_tokens=[token for token in tokens if re.search('\w*',token)]
    minimized_tokens=[w.lower() for w in cleaned_tokens]
    minimized_tokens  
    return minimized_tokens

def tokenize_pages(all_pages):
    tokened_pages=[]
    maximum=len(all_pages)
    i=0
    for char,text in all_pages:
        print(f"\rcalculating {((i*(1/maximum)*100)):5.2f}% completed, please wait", end = ' ')
        tokened_pages.append([char,tokenize_and_lower(text)])
        i+=1
    print(f"\rcalculating {100:5.2f}% completed, Done!               ")
    return tokened_pages

def import_Sentiment_file():
    text_file = open("Data_Set_S1"+".txt", "r", encoding= 'utf-8')
    lines = text_file.readlines()
    lines= [line[:-2] for line in lines[4:]]
    wordset=[]
    for line in lines:
        word=line.split("\t")[0]
        score=float(line.split("\t")[2])
        wordset.append([word,score])
    return wordset


def calculate_sentiment(tokenlist):
    tokenlist= [token.lower() for token in tokenlist] #make lowercase
    lookup_list= import_Sentiment_file()     #get datasetlist
    sentiment=0
    words=0
    distributed_text=nltk.FreqDist(tokenlist)
    for token in distributed_text:
        for (lookupname,value) in lookup_list:
            if (token==lookupname):
                sentiment+=distributed_text[token]*value
                words+=distributed_text[token]
    return sentiment/words
    
def calculate_all_sentiment(universe):
    sentiment=[]
    percent=0.0;
    i=0
    maximum=len(universe)
    for hero,text in universe:
        print(f"\rcalculating {((i*(1/maximum)*100)):5.2f}% completed, please wait", end = ' ')
        sentiment.append([hero,calculate_sentiment(text)])
        i+=1
    print(f"\rcalculating {100:5.2f}% completed, Done!               ")
    save_to_sentiment_file(sentiment,f"sentiment_{universe[0][0]}")
    return sentiment
# save to sentiment file

def save_to_sentiment_file(sentiment_in,filename,path=""):
    string=""
    for name,score in sentiment_in:
        string+=name+"|'|"+str(score)+"|||"
    write_to_file(filename,string,path)
    print(f"sentiment scores saved as {filename}")    
    


#open sentiment from file
def open_sentiment_from_file(filename):
    sentiment_file=open_file(filename)
    string=sentiment_file[0]
    sentiments= string.split("|||")
    sen_from_file=[[name.split("|'|")[0],float(name.split("|'|")[1])] for name in sentiments[:-1]]
    return sen_from_file

def universes_sentiment(universes,sentiment):
    universe_sentiment_scores=[]
    for name,universe in universes:
        universe_sentiment_scores.append([name, [[name,score] for name,score in sentiment if name in universe]] )
    return universe_sentiment_scores

def group_character_with_pages(character_list,path="CharacterPages//"):
    all_pages=[]
    for name in character_list:
        filename = path + name + ".txt"
        try:
            string = open(filename, "r", encoding="utf-8").read()
            words = string
            all_pages.append([name.replace("SEMICOLON",":"),words])
        except:
            continue
            #print(f"Error with {name}")
    return all_pages

def open_file(filename,place=""):
    text_file = open(place+filename.replace(":","SEMICOLON")+".txt", "r", encoding= 'utf-8') # ':' can't be part of filename
    return text_file.readlines()

## Generate list of files

In [3]:
# All characters
t = os.listdir("completelist/")
Characters_pages = [w[0:-4] for w in t]

## Group all characters with their files

In [6]:
all_pages=group_character_with_pages(Characters_pages)

In [7]:
all_pages_tokenized=tokenize_pages(all_pages)

calculating 100.00% completed, Done!                                                                           


In [ ]:
# Warning! takes about 1 hour, Use the next function if this have already run
sentiment=calculate_all_sentiment(all_pages_tokenized)

In [12]:
#Get sentiment from a file
sentiment_from_file=open_sentiment_from_file("sentiment")